# Find precipitation value for pulsed and upslope storms

In [1]:
import numpy as np
import netCDF4
import pandas as pd
import matplotlib.pyplot as plt


np.warnings.filterwarnings('ignore')



In [66]:
m = ['11', '12', '01', '02', '03']

### Minuelty data

In [119]:
#obs_double_fence_acc = []
#obs_double_fence_acc_mean = []
#obs_wind_dir_10m = []
#obs_wind_speed_10m = []
obs_f = dict()
obs_double_fence_acc_mean = dict()
obs_wind_dir_10m   = dict()
obs_wind_speed_10m = dict()
obs_dofe_acc   = dict()
obs_wind_dir   = dict()
obs_wind_speed = dict() 
obs_dofe_acc_rolling = dict()
obs_wind_dir_rolling = dict()
obs_wind_speed_rolling = dict()
obs_diff = dict()
westerly_idx = dict()
easterly_idx = dict()
westerly = dict()
easterly = dict()


for month in m:
    if month == '11':
        t = np.arange(7,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day

        obs_dirnc = '../../Data/Weathermast/Met_%s%s%s_Haukeliseter.nc' %(year,month,day)
        obs_f[int(year+month+day)] = netCDF4.Dataset(obs_dirnc, 'r')

     #   obs_double_fence_acc[int(year+month+day)] = (obs_f[int(year+month+day)].variables['double_fence_accum'][:])
        obs_double_fence_acc_mean[int(year+month+day)] = (np.nanmean(obs_f[int(year+month+day)].variables['double_fence_accum'][:], axis=1))
        obs_wind_dir_10m[int(year+month+day)]   = (obs_f[int(year+month+day)].variables['10m_wind_dir_mast1'][:])
        obs_wind_speed_10m[int(year+month+day)] = (obs_f[int(year+month+day)].variables['10m_wind_speed_mast1'][:])
        
        obs_f[int(year+month+day)].close
        
        ## Create a rolling mean to smoothen out the data
        # attach the last 9 minutes from the previous day to calculate the rolling mean every 10 minutes
        if month != '11' and day == '01':
            if month == '12':
                previous_day = '30'
            elif month == '01' or month == '02':
                previous_day = '31'
            elif month == '03':
                previous_day = '28'
            if month == '01':
                previous_month = '12'
                previous_year = str(int(year)-1)
            elif month == '12':
                previous_month = str(int(month)-1)
                previous_year = year
            else:
                previous_month = '0%s' %(int(month)-1)
                previous_year = year
            
        elif int(day)-1 < 10:
            previous_day = '0%s' %(int(day)-1)
            previous_month = month
            previous_year = year
        else:
            previous_day = '%s' %(int(day)-1)
            previous_month = month
            previous_year = year
        if year == '2016' and month == '11' and day == '07':
            continue
        else:
            pp = (np.concatenate((obs_double_fence_acc_mean[int(previous_year+previous_month+previous_day)][-9:],
                                  obs_double_fence_acc_mean[int(year+month+day)][:])))
            obs_dofe_acc[int(year+month+day)] = (pp)
            wd = (np.concatenate((obs_wind_dir_10m[int(previous_year+previous_month+previous_day)][-9:], 
                                  obs_wind_dir_10m[int(year+month+day)][:])))
            obs_wind_dir[int(year+month+day)] = (wd)
            ws = (np.concatenate((obs_wind_speed_10m[int(previous_year+previous_month+previous_day)][-9:], 
                                  obs_wind_speed_10m[int(year+month+day)][:])))
            obs_wind_speed[int(year+month+day)] = (ws)
        # Calculate the moving average. That is, take
        # the first ten values, average them, 
        # then drop the first and add the eleventh, etc.
        obs_dofe_acc[int(year+month+day)] = pd.DataFrame(np.array(obs_dofe_acc[int(year+month+day)])[:])
        obs_dofe_acc_rolling[int(year+month+day)] = obs_dofe_acc[int(year+month+day)].rolling(window=10,
                                                                                              min_periods= 7,
                                                                                              axis=0).mean()
        obs_dofe_acc_rolling[int(year+month+day)] = obs_dofe_acc_rolling[int(year+month+day)].values
        obs_dofe_acc_rolling[int(year+month+day)] = np.array(obs_dofe_acc_rolling[int(year+month+day)])[9:,:]

        obs_wind_dir[int(year+month+day)] = pd.DataFrame(np.array(obs_wind_dir[int(year+month+day)])[:])
        obs_wind_dir_rolling[int(year+month+day)] = obs_wind_dir[int(year+month+day)].rolling(window=10,
                                                                                              min_periods=7,
                                                                                              axis=0).mean()
        obs_wind_dir_rolling[int(year+month+day)] = obs_wind_dir_rolling[int(year+month+day)].values
        obs_wind_dir_rolling[int(year+month+day)] = np.array(obs_wind_dir_rolling[int(year+month+day)])[9:,:]

        obs_wind_speed[int(year+month+day)] = pd.DataFrame(np.array(obs_wind_speed[int(year+month+day)])[:])
        obs_wind_speed_rolling[int(year+month+day)] = obs_wind_speed[int(year+month+day)].rolling(window=10,
                                                                                                  min_periods=7,
                                                                                                  axis=0).mean()
        obs_wind_speed_rolling[int(year+month+day)] = obs_wind_speed_rolling[int(year+month+day)].values
        obs_wind_speed_rolling[int(year+month+day)] = np.array(obs_wind_speed_rolling[int(year+month+day)])[9:,:]
        
        
        # calculate the precipitation difference, since the minutelty data is accumulated from the beginning
        obs_diff[int(year+month+day)] = np.empty(shape=(obs_dofe_acc_rolling[int(year+month+day)].shape))#/60))
        obs_diff[int(year+month+day)][:] =  np.nan

        for i in np.arange(0,obs_dofe_acc_rolling[int(year+month+day)].shape[0]-1):#/60):
            obs_diff[int(year+month+day)][int(i),:] = obs_dofe_acc_rolling[int(year+month+day)][int(i)+1] - \
                                                      obs_dofe_acc_rolling[int(year+month+day)][int(i)]
                
#        print(year+month+day, np.nansum(obs_diff[int(year+month+day)]))

#    for day in np.arange(8,17):
        
 #       if day < 10:
  #          day = '0%s' %day
   #     else:
    #        day = '%s' %day

        # Find wind regimes and associated precip amount
        IDX = np.arange(obs_wind_dir_rolling[int(year+month+day)].shape[0])

        ## find all westerly regimes (202.5 - 22.5 deg) 
        westerly_idx[int(year+month+day)]  = IDX[np.logical_or(np.logical_and((obs_wind_dir_rolling[int(year+month+day)][:,0] >= 202.5), 
                                                                              (obs_wind_dir_rolling[int(year+month+day)][:,0] <= 360)),
                                                              (obs_wind_dir_rolling[int(year+month+day)][:,0] <22.5))]
        print([int(year+month+day)], westerly_idx[int(year+month+day)])
        ## find all easterly regimes (22.5 - 202.5 deg)
        easterly_idx[int(year+month+day)]  = IDX[np.logical_and((obs_wind_dir_rolling[int(year+month+day)][:,0] >= 22.5),
                                                                (obs_wind_dir_rolling[int(year+month+day)][:,0] < 202.5))]

        ## find indices for < 30 min westerly
        sub_idx = []

        for i in range(0,westerly_idx[int(year+month+day)] .shape[0]-1):
        #for i in range(0,76):
           # print(i)
            if westerly_idx[int(year+month+day)] [i]+1 == westerly_idx[int(year+month+day)] [i+1]:
                continue
                #print( westerlies_idx[i], westerlies_idx[i+1])
            else:
                #print(i)
                sub_idx.append(i)
     #   print(int(year+month+day), sub_idx)


        if len(sub_idx) == 0:
            westerly[int(year+month+day)] = []
            easterly[int(year+month+day)] = []
            
            westerly_duration = westerly_idx[int(year+month+day)][-1] - westerly_idx[int(year+month+day)][0]
            
            if westerly_duration >= 28:
                westerly[int(year+month+day)] = np.concatenate((westerly[int(year+month+day)],
                                                                westerly_idx[int(year+month+day)][:]))
            elif westerly_duration < 28:
                easterly[int(year+month+day)] = np.concatenate((easterly[int(year+month+day)],
                                                    westerly_idx[int(year+month+day)][:]))
        else:
            westerly[int(year+month+day)] = []
            easterly[int(year+month+day)] = []

            for i in np.arange(0,np.array(sub_idx).shape[0]+1):
                if i == 0:
                    westerly_duration = (westerly_idx[int(year+month+day)] [ sub_idx[0]] - \
                                         westerly_idx[int(year+month+day)] [0])
                elif i == np.array(sub_idx).shape[0]:
                    westerly_duration = (westerly_idx[int(year+month+day)] [-1] - \
                                         westerly_idx[int(year+month+day)] [ sub_idx[np.array(sub_idx).shape[0]-1]+1])
                else:
                    westerly_duration = (westerly_idx[int(year+month+day)] [ sub_idx[i]] - \
                                         westerly_idx[int(year+month+day)] [ sub_idx[i-1]+1])
                if westerly_duration >= 28:
                    if i == 0:
                        westerly[int(year+month+day)] = np.concatenate((westerly[int(year+month+day)], 
                                                                        westerly_idx[int(year+month+day)] [ 0: sub_idx[0]+1]))
            #            print(westerly_duration, RR_westerly)
                    elif i == np.array(sub_idx).shape[0]:
                        westerly[int(year+month+day)] = np.concatenate((westerly[int(year+month+day)], 
                                                                        westerly_idx[int(year+month+day)] [ sub_idx[np.array(sub_idx).shape[0]-1]+1 : ]))
             #           print(westerly_duration, RR_westerly)
                    else:
                        westerly[int(year+month+day)] = np.concatenate((westerly[int(year+month+day)], 
                                                                        westerly_idx[int(year+month+day)] [ sub_idx[i-1]+1 : sub_idx[i]+1]))
              #          print(westerly_duration, RR_westerly)
                elif westerly_duration < 28:
                    if i == 0:
                        easterly[int(year+month+day)] = np.concatenate((easterly[int(year+month+day)], 
                                                                        westerly_idx[int(year+month+day)] [ 0: sub_idx[0]+1]))
                    elif i == np.array(sub_idx).shape[0]:
                        easterly[int(year+month+day)] = np.concatenate((easterly[int(year+month+day)],
                                                                        westerly_idx[int(year+month+day)] [ sub_idx[np.array(sub_idx).shape[0]-1]+1 : ]))
                    else:
                   # print(westerly_duration)
                        easterly[int(year+month+day)] = np.concatenate((easterly[int(year+month+day)],
                                                                        westerly_idx[int(year+month+day)] [ sub_idx[i-1]+1 : sub_idx[i]+1]))

#    westerly[int(year+month+day)]  = westerly[int(year+month+day)].astype(int)
 #   easterly[int(year+month+day)]  = easterly[int(year+month+day)].astype(int)

[20161108] [ 154  155  156  162  163  164  165  166  167  168  169  170  171  172  173
  174  175  181  677  678  679 1002 1003 1004 1005 1006 1007 1008 1136 1137
 1138 1139 1140 1141 1142 1143 1144 1145 1146 1147 1148 1149 1150 1151 1152
 1153 1154 1155 1156 1157 1158 1313 1314 1315 1316 1317 1318 1319 1320 1321
 1322 1323 1324 1325 1326 1327 1328 1329 1330 1331 1332 1333 1334 1335 1336
 1337 1338]
[20161109] [315 316 317 318]
[20161110] [ 158  159  160  161  162  163  164  165  166  167  168  169  170  171  172
  173  174  199  200  201  202  203  204  205  206  207  208  209  210  211
  212  213  214  215  216  217  218  219  220  221  222  223  224  225  226
  227  228  229  230  231  232  233  234  235  236  237  238  239  240  241
  242  243  244  245  246  247  248  249  250  251  252  257  258  259  260
  261  262  263  264  265  266  267  268  269  270  271  272  273  274  275
  276  277  278  279  280  281  282  283  284  285  286  287  288  289  290
  291  292  293  294  295

[20161114] [  88   89   90 ..., 1437 1438 1439]
[20161115] [   0    1    2 ..., 1437 1438 1439]
[20161116] [   0    1    2    3    4    5    6    7    8    9   10   11   12   13   14
   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29
   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44
   45   46   47   48   49   50   51   52   53   54   55   56   57   58   59
   60   61   62   63   64   65   66   67   68   69   70   71   72   73   74
   75   76   77   78   79   80   81   82   83   84   85   86   87   88   89
   90   91   92   93   94   95   96   97   98   99  100  101  102  103  104
  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119
  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134
  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149
  150  151  152  153  154  155  156  157  158  159  160  161  162  163  164
  165  166  167  168  169  170  171  172  173  174  175  

IndexError: index -1 is out of bounds for axis 0 with size 0

In [102]:
                i = 0
                if i == 0:
                    westerly_duration = (westerly_idx[int(year+month+day)] [ sub_idx[0]] - \
                                         westerly_idx[int(year+month+day)] [0])
                elif i == np.array(sub_idx).shape[0]:
                    westerly_duration = (westerly_idx[int(year+month+day)] [-1] - \
                                         westerly_idx[int(year+month+day)] [ sub_idx[np.array(sub_idx).shape[0]-1]+1])
                else:
                    westerly_duration = (westerly_idx[int(year+month+day)] [ sub_idx[i]] - \
                                         westerly_idx[int(year+month+day)] [ sub_idx[i-1]+1])

In [ ]:
np.nansum(obs_diff[20161116][easterly_idx,:]) + np.nansum(obs_diff[20161116][westerly_idx,:])

In [ ]:
easterly[20161116].shape[0]+westerly[20161116].shape[0]+easterly_idx.shape[0]

In [ ]:
np.nansum(obs_diff[20161116][easterly_idx]) + np.nansum(obs_diff[20161116][easterly[20161116]])

In [ ]:
np.nansum(obs_diff[20161116][westerly[int(year+month+day)] ])

In [ ]:
np.nansum(obs_diff[20161116][westerly_idx])

In [ ]:
np.nansum(obs_diff[20161116][easterly_idx,:]) + \
np.nansum(obs_diff[20161116][easterly[int(year+month+day)] ,:])       + \
np.nansum(obs_diff[20161116][westerly[int(year+month+day)] ,:])

In [ ]:
        fig = plt.figure(figsize=(8,8))

        fig.text(.0, .5, '10m - Wind direction [$^{o}$]', ha='center', va='center', rotation='vertical', fontsize = 16)

        ax1 = fig.add_subplot(311)
        ax1.plot(obs_wind_dir_rolling[20161116][:,:],'.')
        ax1.hlines(270,0,obs_wind_dir_rolling[20161116].shape[0])
        ax1.hlines(180,0,obs_wind_dir_rolling[20161116].shape[0])
        ax1.hlines(90,0,obs_wind_dir_rolling[20161116].shape[0])
        ax1.hlines(45,0,obs_wind_dir_rolling[20161116].shape[0], linestyles='dashed')
        ax1.hlines(225,0,obs_wind_dir_rolling[20161116].shape[0], linestyles='dashed')

        ax1.fill_between(np.arange(obs_wind_dir_rolling[20161116].shape[0]), 202.5, 360,  facecolor='gray', alpha=0.5)
        ax1.fill_between(np.arange(obs_wind_dir_rolling[20161116].shape[0]), 0, 22.5,  facecolor='gray', alpha=0.5)

        ax1.set_xlim([0, obs_wind_dir_rolling[20161116].shape[0]-1])
        ax1.set_ylim([0, 360])
        ax1.set_xticks(np.arange(0,obs_wind_dir_rolling[20161116].shape[0]+120,120))    
        ax1.set_xticklabels([0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24 ])
        ax1.set_yticks(np.arange(0,405,45))
        fig.suptitle("%s-%s-%s" %(year,month,day), fontsize=14,fontweight='bold')
        ax1.tick_params(labelsize=14)
        plt.show()

In [ ]:
for seconds in westerly:
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)

    print('{:d}:{:02d}:{:02d}'.format(h, m, s)) # Python 3

### Eklima data

### MEPS data